In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
import catboost as cb

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
from joblib import dump, load

In [4]:
train = pd.read_csv('../data/tabular-playground-series-aug-2021/train.csv', index_col='id')

In [5]:
test = pd.read_csv('../data/tabular-playground-series-aug-2021/test.csv', index_col='id')

In [6]:
X = train.drop('loss', axis=1)
y = train.loss

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=4)

In [8]:
lgb = load('../lgb.joblib') 

In [9]:
_ = lgb.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] num_iterations is set=10000, num_trees=10000 will be ignored. Current value: num_iterations=10000
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[LightGBM] [Warning] lambda_l1 is set=0.7, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7
[LightGBM] [Warning] feature_fraction is set=0.25, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.25
[LightGBM] [Warning] lambda_l2 is set=4.4, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.4


In [10]:
lgb_pred = lgb.predict(X_test)

In [11]:
cat = load('../cat.joblib') 

In [12]:
_ = cat.fit(X_train, y_train)

In [13]:
cat_pred = cat.predict(X_test)

In [14]:
def blend(coeff):    
    pred = (lgb_pred * coeff) + (cat_pred * (1.0 - coeff))   
    rmse = np.sqrt(mean_squared_error(y_test, pred)) 
    return rmse

best_blend = min([(blend(0.1 * i), 0.1 * i) for i in range(0, 11)])[1]

In [15]:
np.sqrt(mean_squared_error(y_test, lgb_pred))

7.855417933389205

In [16]:
np.sqrt(mean_squared_error(y_test, cat_pred))

7.856342481453945

In [17]:
blend(best_blend)

7.8496988508336

In [18]:
lgb = load('../lgb.joblib') 

In [19]:
cat = load('../cat.joblib')

In [20]:
test['loss'] = (lgb.predict(test) * best_blend) + (cat.predict(test) * (1.0 - best_blend))

In [21]:
test[['loss']].to_csv('../submission.csv')